In [10]:
import pandas as pd 

SEQLEN = 60 # Every 60 sec
FUTUREPREDICT = 3 # Predict 3 periods
RATIO2PREDICT = "LTC-USD" # What to predict

def classify(current, future):
    if float(future) > float(current):
        return 1
    else: return 0

df = pd.read_csv("crypto_data/LTC-USD.csv", names=['time', 'low', 'high', 'open', 'close', 'volume'])
main_df = pd.DataFrame()

ratios = ['BTC-USD', 'LTC-USD', 'ETH-USD', 'BCH-USD']
for ratio in ratios:
    dataset = f'crypto_data/{ratio}.csv'
    
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]

    if len(main_df) == 0:
        main_df = df
    else: 
        main_df = main_df.join(df)

main_df['future'] = main_df[f"{RATIO2PREDICT}_close"].shift(-FUTUREPREDICT)
main_df['target'] = list(map(classify, main_df[f"{RATIO2PREDICT}_close"], main_df['future']))

print(main_df[[f"{RATIO2PREDICT}_close", "future", 'target']].head)

<bound method NDFrame.head of             LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
...                   ...        ...     ...
1535214960      58.020000  58.020000       0
1535215020      58.009998  58.080002       1
1535215080      58.020000        NaN       0
1535215140      58.020000        NaN       0
1535215200      58.080002        NaN       0

[97724 rows x 3 columns]>
